In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation,Dense,CuDNNLSTM,LSTM,RepeatVector,Permute,multiply,Lambda, Concatenate, Dropout, Embedding, Conv1D,GlobalMaxPooling1D, Flatten ,MaxPooling1D, GlobalAvgPool1D,BatchNormalization
from tensorflow.keras import backend as K


In [0]:
units = 64
max_length = 50
vocab_size = 20000
embedding_size = 20


_input = Input(shape=[max_length], dtype='int32')

# get the embedding layer
embedded = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_size,
        input_length=max_length,
    )(_input)

activations = LSTM(units, return_sequences=True)(embedded)

# compute importance for each step
attention = Dense(1, activation='tanh')(activations) 
attention = Flatten()(attention)
attention = Activation('softmax')(attention)
attention = RepeatVector(units)(attention)
attention = Permute([2, 1])(attention)

# apply the attention
sent_representation = multiply([activations, attention])
sent_representation = Lambda(lambda xin: K.sum(xin, axis=-1))(sent_representation)
#sent_representation = Lambda(lambda xin: K.sum(xin, axis=1))(sent_representation)
#summing up over rows or cols, over rows (axis=1) seems to work better and makes intuitively more sense

probabilities = Dense(10, activation='softmax')(sent_representation)

model = Model(inputs=_input, outputs=probabilities)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[])

In [32]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 50, 20)       400000      input_14[0][0]                   
__________________________________________________________________________________________________
lstm_12 (LSTM)                  (None, 50, 64)       21760       embedding_13[0][0]               
__________________________________________________________________________________________________
dense_17 (Dense)                (None, 50, 1)        65          lstm_12[0][0]                    
____________________________________________________________________________________________